In [224]:
"""
Shows SpamAssassin Python Wrapper at work.

https://pypi.org/project/spamassassin-client/


- SETUP: -
1) Have spamasassin installed
    > sudo apt get spamassassin
2) > pip install spamassassin_client
3) Start spamassassin server
    > sudo spamd
4) Use spamassassin_client

- NOTE: -
REMEMBER TO SHUT DOWN SPAMD SERVER WHEN DONE

"""
import os
from spamassassin_client import SpamAssassin
import pandas as pd
import numpy as np

### Forming any text into valid email including headers and signatures
This is so we can ignore pesky SpamAssassin header and signature rules

In [225]:
# NOTE: Want to prepend this to any text to test that does not already have a header.
#           This will be the case for any text outside of the spam_assassin_corpus
#           This is from the sample-spam.txt file -> does not have any header issues
HAM_HEADER = """Subject: Example header
Message-ID: <GTUBE1.1010101@example.net>
Date: Wed, 23 Jul 2003 23:30:00 +0200
From: Sender <sender@example.net>
To: Recipient <recipient@example.net>
Precedence: junk
MIME-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
"""

header_lines = HAM_HEADER.split('\n')
REQ_HEADERS = {(line.split(':')[0]+':').lower() for line in header_lines if line != ''}
REQ_HEADERS

{'content-transfer-encoding:',
 'content-type:',
 'date:',
 'from:',
 'message-id:',
 'mime-version:',
 'precedence:',
 'subject:',
 'to:'}

In [226]:
def add_header(text):
    # NOTE: only tested on data/spam_ham_dataset.csv
    txt_lines = text.split('\n')
    header_lines = HAM_HEADER.split('\n')
    

In [227]:
spam_ham_df = pd.read_csv('data/spam_ham_dataset.csv')
spam = spam_ham_df[spam_ham_df['label'] == 'spam']
ham = spam_ham_df[spam_ham_df['label'] == 'ham']

spam_sample = spam.iloc[0]
ham_sample = ham.iloc[0]

# NOTE: looking at the format of the data.
#   We can see we have subject lines. Need to fit our header into there
# print("----SPAM----")
# print(spam_sample['text'])
# print("----HAM----")
# print(ham_sample['text'])

In [228]:
print(add_header(ham_sample['text']))

None


In [231]:
with open("data/spam_working_with.txt", 'r') as f:
    spam_text = f.read()
# spam_text = spam_text.lower()
spam_bytes = spam_text.encode('utf-8')
spam_bytes

b"Subject: A CRY FOR HELP\nMessage-ID: <GTUBE1.1010101@example.net>\nDate: Wed, 23 Jul 2003 23:30:00 +0200\nFrom: Sender <sender@example.net>\nTo: Recipient <recipient@example.net>\nPrecedence: junk\nMIME-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\n\nDEAR FRIEND,I AM MRS.  SESE-SEKO WIDOW OF LATE PRESIDENT MOBUTU\nSESE-SEKO OF ZAIRE? NOW KNOWN AS DEMOCRATIC REPUBLIC\nOF CONGO (DRC).  I AM MOVED TO WRITE YOU THIS LETTER,\nTHIS WAS IN CONFIDENCE  CONSIDERING MY PRESENTCIRCUMSTANCE AND SITUATION.\nI ESCAPED ALONG WITH MY HUSBAND AND TWO OF OUR SONS\nGEORGE  KONGOLO  AND BASHER  OUT OF DEMOCRATIC REPUBLIC OF\nCONGO (DRC) TO ABIDJAN, COTE D'IVOIRE WHERE MY FAMILY\nAND I SETTLED, WHILE WE LATER MOVED  TO SETTLED IN\nMORROCO WHERE MY HUSBAND LATER DIED OF CANCER\nDISEASE. HOWEVER DUE TO THIS SITUATION WE DECIDED TO\nCHANGED  MOST OF MY HUSBAND'S BILLIONS OF DOLLARS\nDEPOSITED IN SWISS BANK AND OTHER COUNTRIES INTO OTHER\nFORMS OF MONEY CODED FOR 

In [232]:
assassin = SpamAssassin(spam_bytes)
print(assassin.get_fulltext())

Spam detection software, running on the system "Nicks-MacBook-Pro-8.local",
has identified this incoming email as possible spam.  The original
message has been attached to this so you can view it or label
similar future email.  If you have any questions, see
the administrator of that system for details.

Content preview:  DEAR FRIEND,I AM MRS. SESE-SEKO WIDOW OF LATE PRESIDENT MOBUTU
SESE-SEKO OF ZAIRE? NOW KNOWN AS DEMOCRATIC REPUBLIC OF CONGO (DRC). I AM
MOVED TO WRITE YOU THIS LETTER, THIS WAS IN CONFIDENCE CONSIDER [...]

Content analysis details:   (8.4 points, 5.0 required)

pts rule name              description
---- ---------------------- --------------------------------------------------
-0.0 NO_RECEIVED            Informational: message has no Received headers
0.5 SUBJ_ALL_CAPS          Subject is all capitals
-0.0 NO_RELAYS              Informational: message was not relayed via SMTP
2.6 DEAR_FRIEND            BODY: Dear Friend? That's not very dear!
1.5 NA_DOLLARS          